In [9]:
import pickle

import pandas as pd

df = pd.read_csv('../data/data.csv').rename(columns={'label': 'labels'})

In [10]:
df

,message,labels
0,! У Михаила Гуцериева все-таки появились бабло...,149
1,!? Еще пять компаний группы «Россети» меняют ю...,141
2,!?В правительстве утвердили размещение голосую...,103
3,!?Ожидаем шапито на неделе в нашем цирке народ...,157
4,!?Системно значимых банков всё ещё 11. Изменен...,2;48
...,...,...
16604,🪨 Взгляд на компанию: «Мечел» — эффект от отме...,99
16605,🪨 Взгляд на компанию: «Мечел»: 3-й кв. 2023 г....,99
16606,🪨 Мечел: акции с потенциалом роста свыше 90% д...,99
16607,🪨 Мечел: анализ ключевых тем. Взгляд БКС Мы п...,99


In [41]:
import json
import re
import numpy as np
from abc import abstractmethod


class BaseNERTester:

    @abstractmethod
    def infer_batch(self, input_texts: list[str]) -> list[set[int]]:
        """
        Method to run NER inference

        args:
        - input_texts: list[str] - list of input texts

        returns:
        list[set[int]] - list of sets of recognized entities
        """
        pass


class RegexNERTester(BaseNERTester):

    def __init__(self, path_to_issuers_json: str):
        self.path_to_issuers_json = path_to_issuers_json
        #
        with open(self.path_to_issuers_json, 'rt', encoding='utf-8') as f:
            data = json.loads(f.read())
        for d in data:
            temp_tickers = [i.strip().lower() for i in d['BGTickers']]
            for i in range(len(d['BGTickers'])):
                d['BGTickers'][i] = d['BGTickers'][i].strip().lower()
                if d['BGTickers'][i].endswith(' rx'):
                    d['BGTickers'][i] = d['BGTickers'][i][:-3].strip()
            d['synonims'] = [i.strip().lower() for i in d['synonims']]
        self.issuerid2keys = {
            d['issuerid']: set(d['BGTickers']+d['synonims']+[d['company_name_lower'].strip().lower()])
            for d in data
        }
    
    def infer_batch(self, input_texts: list[str]) -> list[set[int]]:
        res = []
        for text in input_texts:
            text_lower = text.lower()
            res_text = set()
            for issuer_id, keys in self.issuerid2keys.items():
                for key in keys:
                    if key in text_lower:
                        res_text.add(issuer_id)
                        break
            res.append(res_text)
        return res


def run_test(tester: BaseNERTester, df: pd.DataFrame, batch_size: int) -> dict[str, float]:
    texts = df['message']
    results = []
    for batch in np.array_split(texts, batch_size):
        results_batch = tester.infer_batch(texts)
        results.extend(results_batch)
    df['predicted'] = results
    return df

In [60]:
%%time

tester = RegexNERTester('../data/issuers_filtered.json')
predicted_df = run_test(tester, df, 1)

CPU times: total: 766 ms
Wall time: 6.67 s


In [45]:
i = 2
print(predicted_df.iloc[i]['labels'])
print(predicted_df.iloc[i]['predicted'])
print(predicted_df.iloc[i]['message'])

103
{142, 7}
!?В правительстве утвердили размещение голосующих акций «Ростелекома» в рамках допэмиссии по закрытой подписке в пользу ВТБ по цене 93,21 руб. Предполагается, что ВТБ выкупит акции «Ростелекома» на общую сумму 66 млрд руб. - Ъ  Новость вышла через 1 мин после закрытия основной сессии на Московской бирже @finascop


In [44]:
df

,message,labels,predicted
0,! У Михаила Гуцериева все-таки появились бабло...,149,{149}
1,!? Еще пять компаний группы «Россети» меняют ю...,141,"{134, 135, 136, 137, 138, 139, 140, 83, 123, 223}"
2,!?В правительстве утвердили размещение голосую...,103,"{142, 7}"
3,!?Ожидаем шапито на неделе в нашем цирке народ...,157,"{4, 7, 27, 150, 123, 223}"
4,!?Системно значимых банков всё ещё 11. Изменен...,2;48,"{2, 7, 44, 48, 211, 150, 123}"
...,...,...,...
16604,🪨 Взгляд на компанию: «Мечел» — эффект от отме...,99,"{123, 99, 78}"
16605,🪨 Взгляд на компанию: «Мечел»: 3-й кв. 2023 г....,99,"{123, 65, 99, 78}"
16606,🪨 Мечел: акции с потенциалом роста свыше 90% д...,99,"{123, 99, 133, 78}"
16607,🪨 Мечел: анализ ключевых тем. Взгляд БКС Мы п...,99,"{123, 99}"


In [59]:
data_pred = df.to_dict('records')
for row in data_pred:
    true_set = set([int(i) for i in row['labels'].split(';')])
    row['res'] = len(true_set.intersection(row['predicted'])) == len(true_set)
df_res = pd.DataFrame(data_pred)
df_res['res'].sum()/len(df_res)

0.9348545969052923

In [40]:
tester.issuerid2keys

{1: {'derz', 'derzhava', 'derzp', 'держава'},
 2: {'cbom',
  'credit bank',
  'credit bank of moscow',
  'мкб',
  'московский кредитный банк'},
 3: {'rdrb',
  'roads bank',
  'rosdorbank',
  'russian public joint-stock commercial roads bank',
  'дорожный банк',
  'рдбанк',
  'росдорбанк',
  'российский акционерный коммерческий дорожный банк'},
 4: {'alrosa', 'alrs', 'алроса'},
 5: {'avan', 'avangard', 'авангард'},
 6: {'bank "primorye"',
  'bank primorye',
  'pjscb "primorye"',
  'pjscb primorye',
  'primbank',
  'prmb',
  'public joint-stock commercial bank "primorye"',
  'акб "приморье"',
  'акб приморье',
  'банк "приморье"',
  'банк приморье',
  'приморье'},
 7: {'vtb', 'vtbr', 'банк втб', 'втб'},
 8: {'irgz',
  'irkutskenergo',
  'иркутское энергетики и электрификации',
  'иркутскэнерго',
  'иркэнерго'},
 9: {'kazanorgsintez', 'kzos', 'казаньоргсинтез', 'органический синтез'},
 10: {'lenzoloto', 'lnzl', 'лензолото'},
 11: {'rual', 'rusal', 'объединённая компания "русал"', 'русал'}

In [22]:
mentions = pickle.load(open("../data/mentions texts.pickle", "rb"))
mentions.drop(columns=["messageid"], inplace=True)
mentions.drop(index=mentions[mentions.MessageText.str.len() == 0].index, inplace=True)
mentions.drop(index=mentions[mentions.issuerid < 0].index, inplace=True)

In [25]:
mentions.iloc[0]

ChannelID                                             1197210433
issuerid                                                      90
MessageID                                                   5408
DateAdded                                    2021-02-06 01:42:42
DatePosted                                   2020-04-29 07:29:01
MessageText    ?? Фокус недели #ФН  Сегодня ????? ММК опублик...
IsForward                                                  False
Name: 0, dtype: object

In [24]:
mentions['MessageText'].iloc[0]

'?? Фокус недели #ФН  Сегодня ????? ММК опубликует финансовую отчётность за 1 кв. 2020 #MAGN #Отчетность ?????? Магнит представит операционные результаты за 1 кв. 2020 #MGNT #Результаты ?????? Новатэк опубликует финансовую отчётность за 1 кв. 2020 #NVTK #Отчетность ?????? ФРС США представит решение по процентной ставке #ФРС #Макро ?????? Tesla опубликует финансовую отчётность за 1 кв. 2020 #TSLA #Отчетность  ?????? Microsoft опубликует финансовую отчётность за 1 кв. 2020 #MSFT #Отчетность  ?????? Facebook опубликует финансовую отчётность за 1 кв. 2020 #FB #Отчетность  30 апреля ?????? Детский мир опубликует финансовую отчётность по МСФО за 2019 #DSKY #Отчетность ?????? Энел Россия опубликует финансовую отчётность по РСБУ за 1 кв. #ENRU #Отчетность ????? Evraz представит операционные результаты за 1 кв. 2020 #EVR #Результаты ????? НорНикель опубликует производственные результаты за 1 кв. 2020 #GMKN #Результаты ?????? РусГидро представит финансовую отчётность по РСБУ за 1 кв. 2020 #HYDR 